In [45]:
import numpy as np
import matplotlib.pyplot as plt

In [46]:
def l_matrix(n=100):
    result = np.identity(n)*2
    result[1:,:-1] += -np.identity(n-1)
    result[:-1,1:] += -np.identity(n-1)
    return result

In [47]:
the_matrix = l_matrix(100)

In [62]:
def fixed_point_iterations(A, initial_vector, accuracy):
    # A is the matrix
    
    u = np.copy(initial_vector)
    
    for i in range(1000):
        u /= np.linalg.norm(u)
        u = A@u
        
    
    eigenvalue = np.dot(A@u, u)/np.dot(u,u)
    eigenvector = u
    
    return eigenvalue, eigenvector

In [63]:
fixed_point_iterations(the_matrix, np.random.rand(100), 1)

(3.9978279240784107,
 array([-0.01181712,  0.02352317, -0.03500792,  0.04616275, -0.05688147,
         0.06706111, -0.07660264,  0.08541167, -0.09339914,  0.10048191,
        -0.10658332,  0.11163371, -0.1155708 ,  0.1183401 , -0.11989519,
         0.12019795, -0.11921867,  0.11693616, -0.11333775,  0.10841917,
        -0.10218446,  0.09464573, -0.0858229 ,  0.07574333, -0.06444148,
         0.05195843, -0.03834143,  0.02364333, -0.00792211, -0.00875977,
         0.02633591, -0.04473659,  0.06388934, -0.08371956,  0.10415109,
        -0.12510672,  0.1465088 , -0.16827967,  0.19034215, -0.21261993,
         0.23503796, -0.25752278,  0.28000274, -0.30240826,  0.32467193,
        -0.34672869,  0.36851581, -0.3899729 ,  0.4110419 , -0.43166695,
         0.45179425, -0.47137192,  0.49034978, -0.50867917,  0.52631268,
        -0.54320392,  0.55930731, -0.57457784,  0.58897084, -0.60244183,
         0.61494634, -0.62643978,  0.63687741, -0.64621426,  0.65440519,
        -0.66140496,  0.667168

In [60]:
np.linalg.eigh(the_matrix)

(array([9.67435416e-04, 3.86880573e-03, 8.70130406e-03, 1.54602553e-02,
        2.41391205e-02, 3.47295036e-02, 4.72211589e-02, 6.16020016e-02,
        7.78581192e-02, 9.59737849e-02, 1.15931473e-01, 1.37711876e-01,
        1.61293922e-01, 1.86654798e-01, 2.13769968e-01, 2.42613200e-01,
        2.73156590e-01, 3.05370590e-01, 3.39224034e-01, 3.74684172e-01,
        4.11716698e-01, 4.50285786e-01, 4.90354122e-01, 5.31882942e-01,
        5.74832072e-01, 6.19159959e-01, 6.64823720e-01, 7.11779177e-01,
        7.59980905e-01, 8.09382271e-01, 8.59935484e-01, 9.11591634e-01,
        9.64300750e-01, 1.01801184e+00, 1.07267294e+00, 1.12823116e+00,
        1.18463277e+00, 1.24182319e+00, 1.29974710e+00, 1.35834846e+00,
        1.41757058e+00, 1.47735615e+00, 1.53764736e+00, 1.59838585e+00,
        1.65951289e+00, 1.72096932e+00, 1.78269570e+00, 1.84463231e+00,
        1.90671922e+00, 1.96889638e+00, 2.03110362e+00, 2.09328078e+00,
        2.15536769e+00, 2.21730430e+00, 2.27903068e+00, 2.340487